## Part 1: Preprocessing

In [2]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')

display(attrition_df.head())
display(attrition_df.info())

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1470 entries, 0 to 1469
Data columns (total 27 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Age                       1470 non-null   int64 
 1   Attrition                 1470 non-null   object
 2   BusinessTravel            1470 non-null   object
 3   Department                1470 non-null   object
 4   DistanceFromHome          1470 non-null   int64 
 5   Education                 1470 non-null   int64 
 6   EducationField            1470 non-null   object
 7   EnvironmentSatisfaction   1470 non-null   int64 
 8   HourlyRate                1470 non-null   int64 
 9   JobInvolvement            1470 non-null   int64 
 10  JobLevel                  1470 non-null   int64 
 11  JobRole                   1470 non-null   object
 12  JobSatisfaction           1470 non-null   int64 
 13  MaritalStatus             1470 non-null   object
 14  NumCompaniesWorked      

None

In [6]:
# Determine the number of unique values in each column.
display(attrition_df.nunique().sort_values())
attrition_df.columns

,0
Attrition,2
PerformanceRating,2
OverTime,2
MaritalStatus,3
BusinessTravel,3
Department,3
JobSatisfaction,4
RelationshipSatisfaction,4
JobInvolvement,4
EnvironmentSatisfaction,4


Index(['Age', 'Attrition', 'BusinessTravel', 'Department', 'DistanceFromHome',
       'Education', 'EducationField', 'EnvironmentSatisfaction', 'HourlyRate',
       'JobInvolvement', 'JobLevel', 'JobRole', 'JobSatisfaction',
       'MaritalStatus', 'NumCompaniesWorked', 'OverTime', 'PercentSalaryHike',
       'PerformanceRating', 'RelationshipSatisfaction', 'StockOptionLevel',
       'TotalWorkingYears', 'TrainingTimesLastYear', 'WorkLifeBalance',
       'YearsAtCompany', 'YearsInCurrentRole', 'YearsSinceLastPromotion',
       'YearsWithCurrManager'],
      dtype='object')

In [7]:
# Create y_df with the Attrition and Department columns

#  Attrition:  Yes, No **(encode to 1, 0)
#  Department: 3 Research & Development, Sales and ...

y_df = attrition_df[['Attrition', 'Department']]
display(y_df.head())

,Attrition,Department
0,Yes,Sales
1,No,Research & Development
2,Yes,Research & Development
3,No,Research & Development
4,No,Research & Development


In [31]:
# inspect the values of the y columns:

for col in y_df.columns:
    unique_values = sorted(y_df[col].unique().tolist())
    val_type = type(unique_values[0])
    print(f'{col} ({len(unique_values)}) {type(unique_values[0])}: {unique_values}')

Attrition (2) <class 'str'>: ['No', 'Yes']
Department (3) <class 'str'>: ['Human Resources', 'Research & Development', 'Sales']


In [32]:
# look at all X feature columns
X_df = attrition_df.drop(columns = ['Attrition', 'Department'])
display(X_df.head())

,Age,BusinessTravel,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,JobLevel,JobRole,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Travel_Rarely,1,2,Life Sciences,2,94,3,2,Sales Executive,...,3,1,0,8,0,1,6,4,0,5
1,49,Travel_Frequently,8,1,Life Sciences,3,61,2,2,Research Scientist,...,4,4,1,10,3,3,10,7,1,7
2,37,Travel_Rarely,2,2,Other,4,92,2,1,Laboratory Technician,...,3,2,0,7,3,3,0,0,0,0
3,33,Travel_Frequently,3,4,Life Sciences,4,56,3,1,Research Scientist,...,3,3,0,8,3,3,8,7,3,0
4,27,Travel_Rarely,2,1,Medical,1,40,3,1,Laboratory Technician,...,3,4,1,6,3,3,2,2,2,2


In [34]:
# inspect the values, their number, and type for each column (using a loop):

X_vals_str = []
unique_vals_dict_str = {}
X_vals_int = []
unique_vals_dict_int = {}
X_vals = []
unique_vals_dict = {}

for col in X_df.columns:
    unique_values = sorted(X_df[col].unique().tolist())
    val_type = type(unique_values[0])

    if val_type == str:
        X_vals_str.append(col)
        unique_vals_dict_str.update({col : unique_values})
    elif val_type == int:
        X_vals_int.append(col)
        unique_vals_dict_int.update({col : unique_values})
    else:
        X_vals.append(col)
        unique_vals_dict.update({col : unique_values})

print ("\nStrings: ------------------------\n")
print(X_vals_str, "\n")
for key, value in unique_vals_dict_str.items():
    print(f"{key} ({len(value)}) : {value}")

print ("\nInts: ------------------------\n")
print(X_vals_int, "\n")
for key, value in unique_vals_dict_int.items():
    print(f"{key} ({len(value)}) : {value}")

print ("\nOther: ------------------------\n")
print(X_vals, "\n")
for key, value in unique_vals_dict.items():
    print(f"{key} ({len(value)}) : {value}")


Strings: ------------------------

['BusinessTravel', 'EducationField', 'JobRole', 'MaritalStatus', 'OverTime'] 

BusinessTravel (3) : ['Non-Travel', 'Travel_Frequently', 'Travel_Rarely']
EducationField (6) : ['Human Resources', 'Life Sciences', 'Marketing', 'Medical', 'Other', 'Technical Degree']
JobRole (9) : ['Healthcare Representative', 'Human Resources', 'Laboratory Technician', 'Manager', 'Manufacturing Director', 'Research Director', 'Research Scientist', 'Sales Executive', 'Sales Representative']
MaritalStatus (3) : ['Divorced', 'Married', 'Single']
OverTime (2) : ['No', 'Yes']

Ints: ------------------------

['Age', 'DistanceFromHome', 'Education', 'EnvironmentSatisfaction', 'HourlyRate', 'JobInvolvement', 'JobLevel', 'JobSatisfaction', 'NumCompaniesWorked', 'PercentSalaryHike', 'PerformanceRating', 'RelationshipSatisfaction', 'StockOptionLevel', 'TotalWorkingYears', 'TrainingTimesLastYear', 'WorkLifeBalance', 'YearsAtCompany', 'YearsInCurrentRole', 'YearsSinceLastPromotion'

In [ ]:
# check X_df Corrlated Columns




In [ ]:
# check X_df Feature Importance

In [ ]:
# BACKGROUND:
#   'Age',       num values
#  'Education',  1,2,3,4,5
#  'EducationField', 6: Medical, Marketing, Life Sciences, Technical Degree, Other
#  'MaritalStatus', 3 values: single, married, divorced?

# JOB STRESSORS
#   'DistanceFromHome', num value (threshold?)
#   'BusinessTravel' 3: Travel_Frequently, Travel_Rarely, Never Travel?

# LIFE SATISFACTION
#   'RelationshipSatisfaction',  1,2,3,4
#   'WorkLifeBalance',           1,2,3,4
#   'EnvironmentSatisfaction',   1,2,3,4
#   'OverTime',
#   'PercentSalaryHike',

# JOB SATISFACTION/PERFORMANCE/COMPENSATION:
#   'JobSatisfaction',   1,2,3,4
#   'JobInvolvement',    1,2,3,4
#   'JobLevel',
#   'JobRole',
#   'PerformanceRating',
#   'StockOptionLevel',
#   'HourlyRate',
#   'TrainingTimesLastYear',
#
# TENURE
# 'NumCompaniesWorked', 'TotalWorkingYears',
# 'YearsAtCompany', 'YearsInCurrentRole', 'YearsSinceLastPromotion','YearsWithCurrManager'


Education                   int64
Age                         int64
DistanceFromHome            int64
JobSatisfaction             int64
OverTime                   object
StockOptionLevel            int64
WorkLifeBalance             int64
YearsAtCompany              int64
YearsSinceLastPromotion     int64
NumCompaniesWorked          int64
dtype: object

In [ ]:
X_df = attrition_df.drop(columns = ['Attrition', 'Department'])
display(X_df.head())

In [ ]:
# Create a list of at least 10 column names to use as X data
# Create X_df using your selected columns
# Show the data types for X_df

In [ ]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split



In [ ]:
# Convert your X data to numeric data types however you see fit
# Add new code cells as necessary


No     788
Yes    314
Name: OverTime, dtype: int64

In [ ]:
# Create a StandardScaler


# Fit the StandardScaler to the training data


# Scale the training and testing data



In [ ]:
# Create a OneHotEncoder for the Department column


# Fit the encoder to the training data


# Create two new variables by applying the encoder
# to the training and testing data




array([[0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       ...,
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.]])

In [ ]:
# Create a OneHotEncoder for the Attrition column


# Fit the encoder to the training data


# Create two new variables by applying the encoder
# to the training and testing data



array([[1., 0.],
       [0., 1.],
       [1., 0.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]])

## Create, Compile, and Train the Model

In [ ]:
# Find the number of columns in the X training data


# Create the input layer


# Create at least two shared layers


In [ ]:
# Create a branch for Department
# with a hidden layer and an output layer

# Create the hidden layer


# Create the output layer



In [ ]:
# Create a branch for Attrition
# with a hidden layer and an output layer

# Create the hidden layer


# Create the output layer



In [ ]:
# Create the model


# Compile the model


# Summarize the model


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input (InputLayer)          [(None, 10)]                 0         []                            
                                                                                                  
 shared1 (Dense)             (None, 64)                   704       ['input[0][0]']               
                                                                                                  
 shared2 (Dense)             (None, 128)                  8320      ['shared1[0][0]']             
                                                                                                  
 department_hidden (Dense)   (None, 32)                   4128      ['shared2[0][0]']             
                                                                                              

In [ ]:
# Train the model



Epoch 1/100
35/35 [==============================] - 14s 25ms/step - loss: 1.4162 - department_output_loss: 0.8374 - attrition_output_loss: 0.5787 - department_output_accuracy: 0.6316 - attrition_output_accuracy: 0.7024
Epoch 2/100
35/35 [==============================] - 1s 27ms/step - loss: 1.2070 - department_output_loss: 0.7661 - attrition_output_loss: 0.4408 - department_output_accuracy: 0.6570 - attrition_output_accuracy: 0.8403
Epoch 3/100
35/35 [==============================] - 0s 13ms/step - loss: 1.1520 - department_output_loss: 0.7539 - attrition_output_loss: 0.3981 - department_output_accuracy: 0.6570 - attrition_output_accuracy: 0.8403
Epoch 4/100
35/35 [==============================] - 0s 14ms/step - loss: 1.1231 - department_output_loss: 0.7452 - attrition_output_loss: 0.3779 - department_output_accuracy: 0.6588 - attrition_output_accuracy: 0.8475
Epoch 5/100
35/35 [==============================] - 0s 12ms/step - loss: 1.1031 - department_output_loss: 0.7342 - attriti

In [ ]:
# Evaluate the model with the testing data


12/12 [==============================] - 0s 3ms/step - loss: 4.1624 - department_output_loss: 3.1143 - attrition_output_loss: 1.0481 - department_output_accuracy: 0.5272 - attrition_output_accuracy: 0.8261


[4.162380218505859,
 3.114327907562256,
 1.0480519533157349,
 0.5271739363670349,
 0.8260869383811951]

In [ ]:
# Print the accuracy for both department and attrition


Department predictions accuracy: 0.5271739363670349
Attrition predictions accuracy: 0.8260869383811951


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1.
2.
3.